
... ***CURRENTLY UNDER DEVELOPMENT*** ...


## HyCReeW runup estimation 

inputs required: 
  * Nearshore reconstructed historical storms
  * Nearshore reconstructed simulated storms
  * Historical water levels
  * Synthetic water levels 
  * **Expected Sea Level Rise the Site (3 scenarios)**

    
in this notebook:
  * HyCReWW runup estimation of historical and synthetic events **taking into account the 3 SLR scenarios**
  * Extreme value analysis and validation

In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import numpy as np
import pandas as pd
import xarray as xr
from scipy.interpolate import griddata

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.rbf import RBF_Interpolation
from teslakit.mda import Normalize

from teslakit.plotting.extremes import Plot_ReturnPeriodValidation



## Database and Site parameters

In [3]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/albacid/Projects/TeslaKit_projects'

# offshore
db = Database(p_data)
db.SetSite('ROI')

# climate change - S4
db_S4 = Database(p_data)
db_S4.SetSite('ROI_CC_S4')


# reef characteristics
reef_cs = {
    'rslope': 0.0505,
    'bslope': 0.1667,
    'rwidth': 250,
    'cf': 0.0105,
}


# load Hycreww RBF coefficients and sim. variables min. and max.
var_lims, rbf_coeffs = db.Load_HYCREWW()


In [4]:
# hycreww interpolation 

def hycreww_runup(var_lims, rbf_coeffs, dset):
    '''
    Calculates RunUp using hycreww RBFs (level) and linear interpolation (Runup)
    
    var_lims   - hycreww variables min and max limits
    rbf_coeffs - hycreww rbf coefficients
    dset       - input dataset (pandas.DataFrame with "rbf_vns" columns)
    
    '''
    
    # RBF wave conditions 
    rbf_hs = [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5]
    rbf_hs_lo = [0.005, 0.025, 0.05, 0.005, 0.025, 0.05, 0.005, 0.025, 0.05, 0.005, 0.025, 0.05, 0.005, 0.025, 0.05 ]
    rbf_vns = ['level', 'rslope', 'bslope', 'rwidth', 'cf']
    
    # RBF parameters
    ix_sc = [0, 1, 2, 3, 4]
    ix_dr = []
    minis = [var_lims[x][0] for x in rbf_vns]
    maxis = [var_lims[x][1] for x in rbf_vns]
    
    
    # add reef characteristics
    for p in reef_cs.keys():
        dset[p] = reef_cs[p]

    # discard data outside limits
    for vn in var_lims.keys():
        dset = dset[(dset[vn] > var_lims[vn][0]) &(dset[vn] < var_lims[vn][1])]
    
    
    # RBF dataset to interpolate
    ds_in = dset[rbf_vns]

    # normalize data
    ds_nm ,_ ,_ = Normalize(ds_in.values, ix_sc, ix_dr, minis=minis, maxis=maxis)

    # RBF interpolation (with all cases?)
    ru_out = []
    for rc in rbf_coeffs:
        ro = RBF_Interpolation(rc['constant'], rc['coeff'], rc['nodes'], ds_nm.T)
        ru_out.append(ro)
    ru_z = np.array(ru_out)
    
    # RU Linear interpolation (hs, hs_lo -> runup)
    RU = []
    for c, (_, r) in enumerate(dset.iterrows()):
        vq = griddata((rbf_hs, rbf_hs_lo), ru_z[:,c], (r['hs'], r['hs_lo2']), method='linear')
        RU.append(vq)
    RU = np.array(RU)
    
    # store runup alongside input data
    dset_out = dset.copy()
    dset_out['runup'] = dset_out['level'] + RU

    return dset_out


## Hycreww RBF Interpolation: Historical

In [ ]:
# Load complete historical data and nearshore waves
data= db.Load_HIST_Complete_daily() 
waves = db.Load_NEARSHORE_RECONSTRUCTION_HIST_storms()

waves = waves.rename_vars({"Hs": "hs", "Tp": "tp", 'Dir':'dir'})  # rename vars
waves['hs_lo2'] = waves['hs']/(1.5613*waves['tp']**2)             # calc. hs_lo2
waves['level'] = data.sel(time=waves.time).level                  # add level


# calculate runup with hycreww
dset = waves[['hs', 'tp', 'dir', 'level', 'hs_lo2']].to_dataframe()
out_hist = hycreww_runup(var_lims, rbf_coeffs, dset)

# store historical runup
#db.Save_NEARSHORE_RUNUP_HIST(out_hist)


## Hycreww RBF Interpolation: Simulation

In [4]:
# Load SLR file

SLR = xr.open_dataset('SLR.nc')
print(SLR)


<xarray.Dataset>
Dimensions:   (scenario: 3, time: 885360)
Coordinates:
  * time      (time) datetime64[ns] 2000-01-01 ... 2100-12-31T22:58:07.500000256
  * scenario  (scenario) float32 0.5 1.0 1.5
Data variables:
    SLR       (time, scenario) float32 ...


In [5]:
# Load complete simulations data and nearshore waves

n_sims_DWT = 10
data_list = db.Load_SIM_Complete_storms(n_sims=n_sims_DWT)
waves_list = db.Load_NEARSHORE_RECONSTRUCTION_SIM_storms(n_sims=n_sims_DWT)

# iterate storms waves simulations
l_sims = []
for dd, ww in zip(data_list, waves_list):
    
    ww = ww.rename_vars({"Hs": "hs", "Tp": "tp", 'Dir':'dir'})  # rename vars
    ww['hs_lo2'] = ww['hs']/(1.5613*ww['tp']**2)                # calc. hs_lo2
    ww['level'] = dd.sel(time=ww.time).level                    # add level
    
    
    # TODO: mean water level in the tidal gage in the period 1991-2009 for this base leve
    
    
    # TODO: 
    # add SLR for the 3 different scenarios. Add the “parabola” since 2000 to 2100
    # ww['level'] = ww['level'] + SLR
    
    
    
    # calculate runup with hycreww
    dset = ww[['hs', 'tp', 'dir', 'level', 'hs_lo2']].to_dataframe()
    out_sim = hycreww_runup(var_lims, rbf_coeffs, dset)
    
    l_sims.append(out_sim)

    
# store simulation runup
#db.Save_NEARSHORE_RUNUP_SIM(l_sims)
# TODO: save new location

KeyboardInterrupt: 


## Methodology Validation: Annual Maxima

In [ ]:

# compare historical and simulations runup annual maxima
hist_A = out_hist.to_xarray()['runup'].groupby('time.year').max(dim='time')
sim_A = xr.concat([x.to_xarray()['runup'].groupby('time.year').max(dim='time') for x in l_sims], 'n_sim')

# Return Period historical vs. simulations
Plot_ReturnPeriodValidation(hist_A, sim_A);


    